In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision      
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import sys

torch.manual_seed(1)    # reproducible
np.random.seed(seed = 1)

# Hyper Parameters
EPOCH = 20           
BATCH_SIZE = 500
LR = 0.001          

In [2]:
def random_select_data(array,ratio):
    selected_index = np.random.choice(array.shape[0],int(array.shape[0]/ratio),replace=False)
    array_selected = array[selected_index]
    return array_selected
def shuffle_along_axis(a, axis):
    np.random.seed(1)
    idx = np.random.rand(*a.shape).argsort(axis=axis)
    return np.take_along_axis(a,idx,axis=axis)

In [3]:
# Prepare training data and testing data
with open("termite_g2_scaffold_image_dict_scaffold19|size10660809_fragment.pickle", "rb") as handle:
    unserialized_training_data = pickle.load(handle)

In [4]:
# OPTION!!! Prepare random negative data
with open("termite_g2_scaffold_RANDOM_image_array_100000_65_2_10_fragment.pickle", "rb") as handle:
    random_negative_data = pickle.load(handle)

In [5]:
# Make training and testing data
temp_stack_array = np.concatenate([unserialized_training_data[x] for x in unserialized_training_data], 0)
temp_stack_array_read_layer = temp_stack_array[:,1,32]
temp_stack_array_read_layer[temp_stack_array_read_layer != 0].shape
temp_stack_array_label = np.zeros(temp_stack_array.shape[0])
for index in np.arange(0,temp_stack_array.shape[0]):
    if len(temp_stack_array[index,1,:][temp_stack_array[index,1,:] != 0]):
        temp_stack_array_label[index] = 1 
pos_array_ori = temp_stack_array[temp_stack_array_label == 1]
neg_array_ori = temp_stack_array[temp_stack_array_label == 0]

######## Make random selected negative data
#neg_array_ori_selected = random_select_data(neg_array_ori,4)

# OPTION!USE RANDOM DATA
neg_array_ori_selected = random_select_data(random_negative_data,1)
pos_array_ori = random_select_data(pos_array_ori,2)


print("Positive data: ", pos_array_ori.shape)
print("Negative data: ", neg_array_ori_selected.shape)

Positive data:  (59821, 2, 65)
Negative data:  (100000, 2, 65)


In [6]:
POS_NEG_COMBINE_IMAGE = np.concatenate((pos_array_ori,neg_array_ori_selected),axis=0)
POS_LABEL = np.ones(pos_array_ori.shape[0])
NEG_LABEL = np.zeros(neg_array_ori_selected.shape[0])
POS_NEG_COMBINE_LABEL = np.concatenate((POS_LABEL,NEG_LABEL),axis=0)


POS_NEG_COMBINE_IMAGE_shuffle = shuffle_along_axis(POS_NEG_COMBINE_IMAGE, axis = 0)
POS_NEG_COMBINE_LABEL_shuffle = shuffle_along_axis(POS_NEG_COMBINE_LABEL, axis = 0)

# Separate training and testing data
X_train, X_test, y_train, y_test = train_test_split(POS_NEG_COMBINE_IMAGE_shuffle, POS_NEG_COMBINE_LABEL_shuffle, test_size=0.25, random_state=42)
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print("Training image: ", X_train.shape)
print("Training label: ", y_train.shape)
print("Testing image: ", X_test.shape)
print("Testing label: ", y_test.shape)

Training image:  (119865, 1, 2, 65)
Training label:  (119865,)
Testing image:  (39956, 1, 2, 65)
Testing label:  (39956,)


In [7]:
# Numpy to Torch
X_train_torch = torch.from_numpy(X_train)
X_test_torch = torch.from_numpy(X_test)
y_train_torch = torch.from_numpy(y_train)
y_test_torch = torch.from_numpy(y_test)

X_train_torch = X_train_torch.float()
y_train_torch = y_train_torch.long() 

X_test_torch = X_test_torch.float()
y_test_torch = y_test_torch.long()

print("Training image: ", X_train_torch.shape)
print("Training label: ", y_train_torch.shape)
print("Testing image: ", X_test_torch.shape)
print("Testing label: ", y_test_torch.shape)

Training image:  torch.Size([119865, 1, 2, 65])
Training label:  torch.Size([119865])
Testing image:  torch.Size([39956, 1, 2, 65])
Testing label:  torch.Size([39956])


In [8]:
# Make batch
trainloader = torch.utils.data.DataLoader(X_train_torch, batch_size=100, shuffle=False, num_workers=8) 
trainloader_label = torch.utils.data.DataLoader(y_train_torch, batch_size=100, shuffle=False, num_workers=8)

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 28, 28) (1,2,65)
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=16,    # n_filters
                kernel_size=3,      # filter size
                stride=1,           # filter movement/step
                padding=1,      # 如果想要 con2d 出来的图片长宽没有变化, padding=(kernel_size-1)/2 当 stride=1
            ),      # output shape (16, 28, 28)
            nn.ReLU(),    # activation
            nn.MaxPool2d(kernel_size=2),    # 在 2x2 空间里向下采样, output shape (16, 14, 14) (16,1,33)
        )
        self.conv2 = nn.Sequential(  # input shape (16, 14, 14) (16,1,33)
            nn.Conv2d(16, 32, 1, 1, padding=0),  # output shape (32, 14, 14) (32,1,33)
            nn.ReLU(),  # activation
            #nn.MaxPool2d(2),  # output shape (32, 7, 7)
        )
        self.conv3 = nn.Sequential(  # input shape (16, 14, 14) (16,1,33)
            nn.Conv2d(32, 64, 1, 1, padding=0),  # output shape (32, 14, 14) (32,1,33)
            nn.ReLU(),  # activation
            #nn.MaxPool2d(2),  # output shape (32, 7, 7)
        )
        
        self.out = nn.Linear(32 * 1 * 64, 2)   # fully connected layer, output 10 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)   # 展平多维的卷积图成 (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

cnn = CNN()
print(cnn)  # net architecture

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (out): Linear(in_features=2048, out_features=2, bias=True)
)


In [13]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()   # the target label is not one-hotted
correct = 0
total = 0
loss_list = []
accuracy_list = []
TP_list = []
TN_list = []
FP_list = []
FN_list = []


# training and testing
step_length = len(trainloader)

for epoch in range(EPOCH):
    print("epoch: ", epoch)
    LOSS = 0
    ACC = 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    sum_epoch = 0
    for step, (b_x, b_y) in enumerate(zip(trainloader, trainloader_label)):   
        processing = str(round(((step)/ step_length * 100),2)) + "%" +"\r"
        sys.stdout.write(processing)
        sys.stdout.flush()
        
        output = cnn(b_x)               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        
        pred_temp = torch.max(output, 1)[1].data.numpy().squeeze()
        LOSS += loss.item()
        b_y_numpy = b_y.numpy()
        ACC += len(b_y_numpy[b_y_numpy == pred_temp])
        TP += sum((b_y_numpy == 1) & (pred_temp == 1))
        TN += sum((b_y_numpy == 0) & (pred_temp == 0))
        FP += sum((b_y_numpy == 0) & (pred_temp == 1))
        FN += sum((b_y_numpy == 1) & (pred_temp == 0))
        sum_epoch += len(b_y_numpy)
        
        
    print("Loss in epoch", epoch," = ", LOSS/sum_epoch)
    print("Accuracy in epoch", epoch," = ", ACC/sum_epoch)
    print("True positive: ", TP," / ",sum_epoch)
    print("True negative: ", TN," / ",sum_epoch)
    print("False positive: ", FP," / ",sum_epoch)
    print("False negative: ", FN," / ",sum_epoch)
    loss_list.append(LOSS/sum_epoch)
    accuracy_list.append(ACC/sum_epoch)
    TP_list.append(TP)
    TN_list.append(TN)
    FP_list.append(FP)
    FN_list.append(FN)
    
    print("=====================================================")

epoch:  0
Loss in epoch 0  =  0.006632203524806784
Accuracy in epoch 0  =  0.6264047052934552
True positive:  19  /  119865
True negative:  75065  /  119865
False positive:  38  /  119865
False negative:  44743  /  119865
epoch:  1
Loss in epoch 1  =  0.0066102150145616505
Accuracy in epoch 1  =  0.6265548742335127
True positive:  0  /  119865
True negative:  75102  /  119865
False positive:  1  /  119865
False negative:  44762  /  119865
epoch:  2
Loss in epoch 2  =  0.006610405165636062
Accuracy in epoch 2  =  0.6265632169524048
True positive:  2  /  119865
True negative:  75101  /  119865
False positive:  2  /  119865
False negative:  44760  /  119865
epoch:  3
Loss in epoch 3  =  0.006609926006251261
Accuracy in epoch 3  =  0.6265632169524048
True positive:  0  /  119865
True negative:  75103  /  119865
False positive:  0  /  119865
False negative:  44762  /  119865
epoch:  4
Loss in epoch 4  =  0.006609927044042891
Accuracy in epoch 4  =  0.6265632169524048
True positive:  0  /  1

KeyboardInterrupt: 

In [ ]:
test_output = cnn(X_test_torch)
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_y)

In [19]:
pred_y[pred_y==1].shape

(54911,)